<div >
    <center> <font size=7 color=red>
    <b>Transfer learning Implementation </b></font>
    </center>
    <br>
    
</div>


<div >
    <center> <font size=6><b>Without Fine-Tuning </b></font>
    </center>
    <br>
    
</div>


<div >
    <h1>Import the libraries
    </h1>
</div>


In [2]:
import wandb
from wandb.integration.keras import WandbMetricsLogger
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

<div >
    <h1>Sweep configuration
    </h1>
</div>


In [1]:
sweep_config = {
    'method' : 'grid',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters':{
        'batch_size': {'values': [8]},
        'learning_rate': {'values': [0.0001]},
        'img_size':{'values':[128]},
        'epochs': {'values': [10]},
        'experiment': {'values': ['transfer_learning']}
    }
}


<div >
    <h1>Initialize W&B sweep
    </h1>
</div>


In [3]:
sweep_id = wandb.sweep(sweep_config,project="5-flowers-classification_Transfer_learning_withoutFineTunning")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: oy1s49m1
Sweep URL: https://wandb.ai/ayoubaamir0201-ensam-casablanca/5-flowers-classification_Transfer_learning_withoutFineTunning/sweeps/oy1s49m1


<div >
    <h1>Train function
    </h1>
</div>


In [4]:
def train():
  with wandb.init() as run:
    config = wandb.config

    IMG_HEIGHT = config.img_size
    IMG_WIDTH = config.img_size
    IMG_CHANNELS = 3
    CLASS_NAMES =["daisy", "dandelion", "roses", "sunflowers", "tulips"]

    def read_and_decode(filename, resize_dims):
      img_bytes = tf.io.read_file(filename)
      img = tf.image.decode_jpeg(img_bytes, channels = IMG_CHANNELS)
      img = tf.image.convert_image_dtype(img, tf.float32)
      img = tf.image.resize(img, resize_dims)
      return img

    def parse_csvline(csv_line):
      record_default = ["",""]
      filename, label_string = tf.io.decode_csv(csv_line, record_default)
      img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
      label = tf.where(tf.equal(CLASS_NAMES, label_string))[0,0]
      return img, label

    # Prepare datasets
    train_dataset = (
            tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
            .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(config.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )

    eval_dataset = (
            tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
            .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(config.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )


    #Load pretrained model without the classification head
    base_model = tf.keras.applications.MobileNetV2(
        input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
        include_top = False,  #Remove ImageNet HEAD
        weights = "imagenet" #Use pretrained weights
    )


    #Freeze the base model
    base_model.trainable = False


    #Add a new classification head
    model = keras.Sequential([
        base_model,
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(len(CLASS_NAMES),activation = "softmax")
              ])

    # Compile model
    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = config.learning_rate),
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits = False),
        metrics = ["accuracy"]
    )

    # Visualize model
    model.summary()

    keras.utils.plot_model(
        model,
        show_shapes = True,
        show_layer_names = True,
        to_file = "Visualize_model.png"
    )

    # Train
    callbacks = [WandbMetricsLogger(log_freq = 5)]
    model.fit(
        train_dataset,
        validation_data = eval_dataset,
        epochs = config.epochs,
        callbacks = callbacks
    )


<div >
    <h1>Launch W&B agent
    </h1>
</div>


In [5]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 4bgldq3w with config:
wandb: 	batch_size: 8
wandb: 	epochs: 10
wandb: 	experiment: transfer_learning
wandb: 	img_size: 128
wandb: 	learning_rate: 0.0001
wandb: Currently logged in as: ayoubaamir0201 (ayoubaamir0201-ensam-casablanca) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │         6,405 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,269,509 (8.66 MB)

...

Epoch 1/10
    413/Unknown 110s 226ms/step - accuracy: 0.4199 - loss: 1.5622

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


413/413 ━━━━━━━━━━━━━━━━━━━━ 138s 292ms/step - accuracy: 0.4202 - loss: 1.5615 - val_accuracy: 0.6730 - val_loss: 0.8962
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 93s 226ms/step - accuracy: 0.6923 - loss: 0.8377 - val_accuracy: 0.7676 - val_loss: 0.6804
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 82s 199ms/step - accuracy: 0.7625 - loss: 0.6721 - val_accuracy: 0.8000 - val_loss: 0.5931
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 84s 203ms/step - accuracy: 0.7857 - loss: 0.5869 - val_accuracy: 0.8054 - val_loss: 0.5457
Epoch 5/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 142s 345ms/step - accuracy: 0.8074 - loss: 0.5302 - val_accuracy: 0.8216 - val_loss: 0.5155
Epoch 6/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 203s 347ms/step - accuracy: 0.8275 - loss: 0.4875 - val_accuracy: 0.8270 - val_loss: 0.4945
Epoch 7/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 202s 346ms/step - accuracy: 0.8430 - loss: 0.4529 - val_accuracy: 0.8324 - val_loss: 0.4790
Epoch 8/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 85s 205ms/step - accuracy: 0.8537 - loss: 0.4236 -

batch/accuracy,▁▁▃▃▃▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████
batch/batch_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,██▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁▅▆▆▇▇▇███
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▄▃▃▂▂▂▁▁▁
epoch/val_accuracy,▁▅▆▆▇▇▇███
epoch/val_loss,█▅▃▂▂▂▁▁▁▁
batch/accuracy,0.88017


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.



<div >
    <h1>Big improvements with transfer learning without fine-tuning: val_accuracy ~85%, with corresponding gains in training accuracy
    and reduced loss compared to our earlier results.
    </h1>
    
</div>

<div >
    <h1>Flowers-Class Transfer learning without Fine-tuning experimental report:<a></a>
    </h1>
    
</div>

**https://api.wandb.ai/links/ayoubaamir0201-ensam-casablanca/mdijm9ax**